In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 랜덤 데이터 생성
np.random.seed(42)
data_size = 1000

# 조건 생성
time = np.random.randint(0, 24, data_size)  # 시간 (0~23)
wind_speed = np.random.randint(0, 11, data_size)  # 풍속 (0~10)
cloud_cover = np.random.randint(0, 3, data_size)  # 구름양 (0, 1, 2)
precipitation = np.random.randint(0, 11, data_size)  # 강수량 (0~10)

# 결과 생성: 날씨 좋음(1) 또는 나쁨(0) 가정 규칙
# 날씨가 좋음: 풍속 < 5, 구름양 < 2, 강수량 == 0
result = ((wind_speed < 5) & (cloud_cover < 2) & (precipitation < 2)).astype(int)

# 데이터프레임 생성
data = pd.DataFrame({
    # "Time": time,
    "WindSpeed": wind_speed,
    "CloudCover": cloud_cover,
    "Precipitation": precipitation,
    "WeatherGood": result
})


# 데이터 준비
# X = data[["Time", "WindSpeed", "CloudCover", "Precipitation"]].values  # 입력 데이터
X = data[["WindSpeed", "CloudCover", "Precipitation"]].values  # 입력 데이터
y = data["WeatherGood"].values  # 결과값 (0: 나쁨, 1: 좋음)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tensor 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)  # (N, 1) 형태
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# RNN 입력 형태로 변환 (배치, 타임스텝, 특징)
X_train_rnn = X_train_tensor.unsqueeze(1)  # (batch_size, seq_length=1, input_size=4)
X_test_rnn = X_test_tensor.unsqueeze(1)


ModuleNotFoundError: No module named 'torch'

In [33]:
class WeatherRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(WeatherRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size)  # 초기 hidden state
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])  # 마지막 타임스텝의 출력
        return self.sigmoid(out)


In [34]:
# 모델 초기화
# input_size = 4  # Time, WindSpeed, CloudCover, Precipitation
input_size = 3  # WindSpeed, CloudCover, Precipitation
hidden_size = 16
output_size = 1  # WeatherGood
model = WeatherRNN(input_size, hidden_size, output_size)

# 손실 함수와 옵티마이저
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 학습
epochs = 500
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_rnn)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')


Epoch [100/500], Loss: 0.0684
Epoch [200/500], Loss: 0.0577
Epoch [300/500], Loss: 0.0448
Epoch [400/500], Loss: 0.0376
Epoch [500/500], Loss: 0.0318


In [35]:
# 모델 평가
model.eval()
with torch.no_grad():
    y_pred = model(X_test_rnn)
    y_pred_binary = (y_pred > 0.5).float()  # 0.5 기준 이진화

    # 정확도 계산
    accuracy = (y_pred_binary == y_test_tensor).sum().item() / y_test_tensor.size(0)
    print(f'Accuracy: {accuracy:.4f}')


Accuracy: 0.9950


In [36]:
# 새로운 데이터로 추론
# new_data = torch.tensor([[12, 3, 1, 0],  # 시간: 12시, 풍속: 3, 구름양: 1, 강수량: 0
#                          [18, 8, 2, 5],  # 시간: 18시, 풍속: 8, 구름양: 2, 강수량: 5
#                          [6, 1, 0, 0]],  # 시간: 6시, 풍속: 1, 구름양: 0, 강수량: 0
#                         dtype=torch.float32).unsqueeze(1)  # (batch_size, seq_length=1, input_size=4)
new_data = torch.tensor([[3, 1, 0],  # 풍속: 3, 구름양: 1, 강수량: 0
                         [8, 2, 5],  # 풍속: 8, 구름양: 2, 강수량: 5
                         [1, 0, 0]],  # 풍속: 1, 구름양: 0, 강수량: 0
                        dtype=torch.float32).unsqueeze(1)  # (batch_size, seq_length=1, input_size=4)

with torch.no_grad():
    predictions = model(new_data)
    binary_predictions = (predictions > 0.5).float()
    # print("Predictions:", predictions.squeeze().numpy()
    # # 확률(%)로 변환
    percentages = predictions.squeeze().numpy() * 100

    # 출력
    for i, p in enumerate(percentages):
        print(f"Sample {i + 1}: {p:.2f}% chance of good weather")
    print("Binary Predictions (Good Weather):", binary_predictions.squeeze().numpy())


Sample 1: 84.57% chance of good weather
Sample 2: 0.00% chance of good weather
Sample 3: 99.91% chance of good weather
Binary Predictions (Good Weather): [1. 0. 1.]
